# EDA

In this EDA, the study dataset are examined in more detail. The goal is to understand the data and the structure of it better.


In [33]:
import pandas as pd
import os
import plotly.express as px
import re
import numpy as np
import plotly.graph_objs as go
from IPython.display import display
from IPython.display import Markdown as md
import plotly.io as pio
from pandas_profiling import ProfileReport
from globalvars import *
from functions import *
import kaleido

### Load Data

In [2]:
## Import dataframe
filelocation = 'data/DataClean'
df = pd.read_feather(filelocation)

## load config file
config = pd.read_excel('config/config.xlsx',sheet_name='fragecodes')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230892 entries, 0 to 230891
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   participant_id       230892 non-null  int64         
 1   u_date               230892 non-null  datetime64[ns]
 2   Kommentar            62303 non-null   object        
 3   wime_personal        79308 non-null   float64       
 4   wime_komfort         179566 non-null  float64       
 5   wime_sauberkeit      182781 non-null  float64       
 6   wime_puenktlich      183408 non-null  float64       
 7   wime_platzangebot    184204 non-null  float64       
 8   wime_gesamtzuf       191561 non-null  float64       
 9   wime_preis_leistung  215356 non-null  float64       
 10  wime_fahrplan        222515 non-null  float64       
 11  wime_oes_fahrt       175410 non-null  float64       
 12  S_sprache            230892 non-null  category      
 13  S_alter       

## EDA

Possiblity to filter on a specific timerange. The filter is applied the whole EDA.

In [7]:
startdate = '2019-01-01'
enddate = '2022-10-31'

# Apply time selection for all DataFrames
df= df[(df['u_date'] > startdate) & (df['u_date'] < enddate)]

### Overview

#### General DataFrame Overview

In [8]:
df = df.sort_values("u_date",ascending=False)

In [9]:
df.head(5)

,participant_id,u_date,Kommentar,wime_personal,wime_komfort,wime_sauberkeit,wime_puenktlich,wime_platzangebot,wime_gesamtzuf,wime_preis_leistung,...,ft_vm_kurz,ft_zielort_uic,fg_abfahrt,fg_ankunft,fg_startort_uic,fg_zielort_uic,fg_startort,fg_zielort,ft_startort,ft_zielort
226335,583619,2022-10-30,None,NaN,25.0,75.0,75.0,100.0,100.0,100.0,...,IC,8500113,2022-11-22 16:49:00,2022-11-22 18:10:00,8504300,8500117,Biel/Bienne,Aesch BL,Biel/Bienne,Laufen
228498,586617,2022-10-30,None,100.0,75.0,75.0,75.0,75.0,100.0,100.0,...,IC,8507483,2022-11-22 07:56:00,2022-11-22 10:02:00,8500010,8507483,Basel SBB,Spiez,Basel SBB,Spiez
228374,586445,2022-10-30,None,75.0,100.0,100.0,100.0,100.0,100.0,100.0,...,IR,8505113,2022-11-22 15:28:00,2022-11-22 17:27:00,8503102,8505892,Erlenbach ZH,"Altdorf UR, Telldenkmal",Zürich HB,Altdorf UR
228373,586444,2022-10-30,None,NaN,100.0,100.0,75.0,100.0,100.0,100.0,...,IR,8505000,2022-11-22 15:11:00,2022-11-22 17:30:00,8502007,8505300,Sursee,Lugano,Sursee,Luzern
226237,583484,2022-10-30,None,NaN,NaN,NaN,NaN,NaN,NaN,100.0,...,NaN,NaN,NaT,NaT,NaN,NaN,Solothurn,Olten,Solothurn,Olten


In [10]:
df.shape

(226537, 42)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226537 entries, 226335 to 913
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   participant_id       226537 non-null  int64         
 1   u_date               226537 non-null  datetime64[ns]
 2   Kommentar            61160 non-null   object        
 3   wime_personal        77463 non-null   float64       
 4   wime_komfort         176142 non-null  float64       
 5   wime_sauberkeit      179305 non-null  float64       
 6   wime_puenktlich      179916 non-null  float64       
 7   wime_platzangebot    180701 non-null  float64       
 8   wime_gesamtzuf       188063 non-null  float64       
 9   wime_preis_leistung  211236 non-null  float64       
 10  wime_fahrplan        218347 non-null  float64       
 11  wime_oes_fahrt       172092 non-null  float64       
 12  S_sprache            226537 non-null  category      
 13  S_alter     

In [12]:
## Add additional date columns for easier vizualization
df.insert(loc=2, column = 'year', value=df['u_date'].dt.year) #create additional year col for viz
df.insert(loc=3, column = 'month', value=df['u_date'].dt.month) #create additional month col for viz
df['yearmonth'] = pd.to_datetime(df[['year', 'month']].assign(DAY=1)) #create additional yearmonth col for viz

### TextData

In [13]:

df_text = df.dropna(subset=["Kommentar"])

#### Number of comments per month

In [14]:
df_text_counts = pd.DataFrame(df_text.groupby(['yearmonth'])['participant_id'].count()).reset_index()
df_participantstotals_counts = pd.DataFrame(df.groupby(['yearmonth'])['participant_id'].count()).reset_index()

In [15]:
# Merge Dataframes
df_counts_month = df_text_counts.merge(df_participantstotals_counts, on='yearmonth', how='left')
df_counts_month.rename(columns = {'participant_id_x':'Anzahl Kommentare','participant_id_y':'Anzahl Befragte'}, inplace = True)
df_counts_month['Kommentar Ratio in %'] = (df_counts_month['Anzahl Kommentare']/df_counts_month['Anzahl Befragte'])*100

In [16]:
## plot settings
color_discrete_sequence = color_discrete_sequence #imported from globalvars.py
template = template ##imported from globalvars.py

In [34]:
fig = px.line(df_counts_month,
              x="yearmonth",
              y=['Anzahl Kommentare', 'Anzahl Befragte'],
              color_discrete_sequence=color_discrete_sequence,
              template=template)

fig.update_layout(
    title="Anzahl Befragte und Anzahl Kommentare im Vergleich",
    yaxis_title='Anzahl',
    xaxis_title='Monat',
    height=300,
    legend_title="")

fig.update_traces(line_width=2)
fig.update_traces(patch={"line": {"dash": 'dot'}}, selector={"legendgroup": "Anzahl Befragte"}) 

fig.show()

pio.write_image(fig,"images/fig-anteil_befragte_und_anzahl_kommentare.svg") #save fig as png

ValueError: Failed to start Kaleido subprocess. Error stream:

/Users/dominik/Documents/Masterarbeit Code/nlp-satisfaction/venv/lib/python3.9/site-packages/kaleido/executable/kaleido: line 4: cd: /Users/dominik/Documents/Masterarbeit: No such file or directory
/Users/dominik/Documents/Masterarbeit Code/nlp-satisfaction/venv/lib/python3.9/site-packages/kaleido/executable/kaleido: line 5: ./bin/kaleido: No such file or directory


In [32]:
fig = px.line(df_counts_month,
              x="yearmonth",
              y='Kommentar Ratio in %',
              color_discrete_sequence=color_discrete_sequence,
              template=template)
fig.update_layout(
    title="Anteil Befragte mit Kommentar",
    yaxis_title='Anteil in %',
    xaxis_title='Monat',
    height=300
    )
fig.update_traces(line_width=2)
#fig.update_yaxes(range=(0,40))

# show legend for singel line plots
fig['data'][0]['showlegend'] = True
fig['data'][0]['name'] = 'Anteil Kommentare in %'

fig.show()

pio.write_image(fig,"images/fig-anteil-befragte-mit-kommentar.svg") #save fig as png

ValueError: Failed to start Kaleido subprocess. Error stream:

/Users/dominik/Documents/Masterarbeit Code/nlp-satisfaction/venv/lib/python3.9/site-packages/kaleido/executable/kaleido: line 4: cd: /Users/dominik/Documents/Masterarbeit: No such file or directory
/Users/dominik/Documents/Masterarbeit Code/nlp-satisfaction/venv/lib/python3.9/site-packages/kaleido/executable/kaleido: line 5: ./bin/kaleido: No such file or directory


In [17]:
df_counts_month.describe()

,Anzahl Kommentare,Anzahl Befragte,Kommentar Ratio in %
count,45.000000,45.000000,45.000000
mean,1359.111111,5034.155556,26.800624
std,524.873501,1801.099903,2.518035
min,125.000000,456.000000,22.506454
25%,990.000000,3952.000000,24.677835
50%,1325.000000,5097.000000,26.476602
75%,1703.000000,5875.000000,28.435465
max,2698.000000,9898.000000,31.907954


#### Minimal, maximal and average length of Kommentar

In [18]:
## Add basic text features
df_text["Kommentar"] = remove_redundant_whitespaces(df_text["Kommentar"]) #note: imported function "remove_redundant_whitespaces"
df_text = add_basic_textfeatures(df_text,"Kommentar")

/var/folders/zg/d4glzd_11n11pmq9m6l8yksm0000gn/T/ipykernel_31287/952732434.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [19]:
df_text_average_tokens_median_per_month = pd.DataFrame(df_text.groupby(['yearmonth'])['Kommentar_Tokens'].median()).reset_index()
df_text_average_tokens_mean_per_month = pd.DataFrame(df_text.groupby(['yearmonth'])['Kommentar_Tokens'].mean()).reset_index()

In [20]:
# Merge Dataframes
df_text_tokens_counts_month = df_text_average_tokens_median_per_month.merge(df_text_average_tokens_mean_per_month, on='yearmonth', how='left')
df_text_tokens_counts_month.rename(columns = {'Kommentar_Tokens_x':'Median Tokens','Kommentar_Tokens_y':'Mittelwert Tokens'}, inplace = True)

In [19]:
fig = px.line(df_text_tokens_counts_month,
              x="yearmonth",
              y=['Median Tokens','Mittelwert Tokens'],
              color_discrete_sequence=color_discrete_sequence,
              template=template)
fig.update_layout(
    title="Freitextkommentare: Token-Median & Token-Mittelwert in Vergleich",
    yaxis_title='Anzahl',
    xaxis_title='Monat',
    height=300,
    legend_title="")
fig.update_traces(line_width=2)
fig.update_yaxes(range=(0,35))


fig.show()

pio.write_image(fig,"images/fig-token-median-mittelwert.svg") #save fig as png

NameError: name 'df_text_tokens_counts_month' is not defined

In [22]:
fig = px.histogram(
    df_text,
    x="Kommentar_Tokens",
    range_x = [0,150],
    title="Freitextkommentare: Häufigkeitsverteilung Tokens",
    #color = "year",
    labels={'Kommentar_Tokens':'Anzahl Tokens'},
    color_discrete_sequence=color_discrete_sequence,
    height=300,
    template=template)
    
fig.update_layout(yaxis_title="Anzahl Kommentare")

# Overlay both histograms
#fig.update_layout(barmode='overlay')

# Reduce opacity to see both histograms
#fig.update_traces(opacity=0.5)

fig.show()

pio.write_image(fig,"images/fig-token-hist.svg") #save fig as png

In [23]:
df_text.describe()

,participant_id,year,month,wime_personal,wime_komfort,wime_sauberkeit,wime_puenktlich,wime_platzangebot,wime_gesamtzuf,wime_preis_leistung,wime_fahrplan,S_alter,u_preis,Kommentar_Character,Kommentar_Tokens,Kommentar_Types,Kommentar_TTR
count,61160.000000,61160.000000,61160.000000,22619.000000,48970.000000,49734.000000,49907.000000,50202.000000,52247.000000,57243.000000,60302.000000,61008.000000,53669.000000,61160.000000,61160.000000,61160.000000,61083.000000
mean,360541.658192,2020.352322,6.526897,88.179186,74.256234,76.602916,86.984513,75.499150,78.730996,67.822606,78.240587,51.917880,43.719585,173.376619,26.893035,23.980592,94.696425
std,175914.277284,1.180280,3.296238,19.900731,25.702345,23.184245,24.392820,30.271958,23.354511,29.247308,27.671276,15.583001,276.397018,181.910990,28.603849,22.566264,6.905733
min,41305.000000,2019.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,33.333333
25%,120252.750000,2019.000000,4.000000,77.777778,55.555556,66.666667,77.777778,55.555556,75.000000,50.000000,66.666667,41.000000,8.800000,58.000000,9.000000,8.000000,90.697674
50%,406374.500000,2020.000000,7.000000,100.000000,75.000000,77.777778,100.000000,88.888889,77.777778,75.000000,88.888889,54.000000,17.000000,119.000000,18.000000,17.000000,100.000000
75%,501831.250000,2021.000000,9.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,64.000000,30.600000,219.000000,34.000000,32.000000,100.000000
max,587992.000000,2022.000000,12.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,98.000000,6300.000000,2047.000000,347.000000,230.000000,100.000000


- Median of 122 Characters per Comment (140 original Twitter length)
- Median of 19 Tokens per comment.
- Median TTR of 98.

In [24]:
# Pandas Profiling TextData
profile = ProfileReport(
    df_text[
        [
            #"participant_id",
            #"u_date",
            "Kommentar",
            "Kommentar_Character",
            "Kommentar_TTR",
            "Kommentar_Tokens",
            "Kommentar_Types",
        ]
    ],
    title="TextData",
    lazy=False,
    dark_mode=True,
)

profile.to_file("html/ProfilingTextData.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]/Users/dominik/Documents/nlp-satisfaction/venv/lib/python3.9/site-packages/multimethod/__init__.py:315: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

Summarize dataset:  44%|████▍     | 8/18 [00:05<00:05,  1.74it/s, Calculate spearman correlation]       /Users/dominik/Documents/nlp-satisfaction/venv/lib/python3.9/site-packages/multimethod/__init__.py:315: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 267.60it/s]


### Satisfaction Data

In [51]:
satisfaction_codes = config[config['vartype'] == 'satisfaction']['fragecode']

In [52]:
# Pandas Profiling TextData
profile = ProfileReport(
    df[
        satisfaction_codes
    ],
    title="SatisfactionData",
    lazy=False,
    dark_mode=True,
)

profile.to_file("html/ProfilingSatisfactionData.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]/Users/dominik/Documents/nlp-satisfaction/venv/lib/python3.9/site-packages/multimethod/__init__.py:315: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

Summarize dataset:  64%|██████▎   | 14/22 [00:03<00:02,  3.17it/s, Calculate kendall correlation]       /Users/dominik/Documents/nlp-satisfaction/venv/lib/python3.9/site-packages/scipy/stats/_stats_py.py:5218: RuntimeWarning:

overflow encountered in long_scalars

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 83.82it/s]


In [53]:
df_satisfaction = pd.melt(df, id_vars='yearmonth', value_vars=satisfaction_codes) #wide to long and only keep satisfaction questions
df_satisfaction = df_satisfaction.groupby(['yearmonth', 'variable'])[['value']].mean().reset_index() #groupy monthyear and variable

In [59]:
fig = px.line(
    df_satisfaction,
    x="yearmonth",
    y="value",
    color="variable",
    color_discrete_sequence=color_discrete_sequence,
    template=template,
)

fig.update_layout(
    title="Zufriedenheit mit verschiedenen Aspekten im zeitlichen Verlauf",
    yaxis_title="",
    xaxis_title="",
    height=300,
    legend_title="",
)

fig.update_traces(line_width=2)
fig.update_yaxes(range=(65, 100))
fig.add_vrect(
    x0="2020-03-16",
    x1="2022-03-31",
    line_width=0,
    fillcolor="#0B1F26",
    opacity=0.05,
    annotation_text="Covid",
    annotation_position="top left",
    annotation_font_size=10,
    annotation_font_color="#0B1F26",
)

line_width = 2

fig.show()

pio.write_image(fig, "images/fig-satisfaction_zeitreihe.svg")  # save fig as png


### MetaData

In [63]:
# Pandas Profiling TextData
profile = ProfileReport(
    df[
        [
            "participant_id",
            "u_date",
            "S_alter",
            "S_sex",
            "S_wohnsitz",
            "u_klassencode",
            "u_ga",
            "S_AB3_HTA",
            "R_anschluss",
            "R_stoerung",
            "device_type",
            "dispcode",
            "u_ticket",
            "u_fahrausweis",
            "u_preis",
            "R_zweck",
            "ft_abfahrt",
            "ft_ankunft",
            "ft_startort_uic",
            "ft_tu",
            "ft_vm",
            "ft_vm_kurz",
            "ft_zielort_uic",
            "fg_abfahrt",
            "fg_ankunft",
            "fg_startort_uic",
            "fg_zielort_uic",
            "fg_startort",
            "fg_zielort",
            "ft_startort",
            "ft_zielort",
        ]
    ],
    title="MetaData",
    lazy=False,
    dark_mode=True,
)

profile.to_file("html/ProfilingMetaData.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]/Users/dominik/Documents/nlp-satisfaction/venv/lib/python3.9/site-packages/multimethod/__init__.py:315: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

Summarize dataset:  75%|███████▌  | 33/44 [00:04<00:01,  7.80it/s, Calculate auto correlation]       /Users/dominik/Documents/nlp-satisfaction/venv/lib/python3.9/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning:

The input array could not be properly checked for nan values. nan values will be ignored.

/Users/dominik/Documents/nlp-satisfaction/venv/lib/python3.9/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning:

The input array could not be properly checked for nan values. nan values will be ignored.

/Users/dominik/Documents/nlp-satisfaction/venv/lib/python3.9/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning:

The input array could not be properly checked for nan values. nan values will be ignored.